In [0]:
import torch
from torch import nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torch.nn.functional as f
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize
%matplotlib inline

INPUT_SIZE = 24

def weibull_loglik_discrete(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    hazard0 = torch.pow((y_ + 1e-35) / a_, b_)
    hazard1 = torch.pow((y_ + 1) / a_, b_)

    return -1 * torch.mean(u_ * torch.log(torch.exp(hazard1 - hazard0) - 1.0) - hazard1)


def weibull_loglik_continuous(y_true, ab_pred, name=None):
    y_ = y_true[:, 0]
    u_ = y_true[:, 1]
    a_ = ab_pred[:, 0]
    b_ = ab_pred[:, 1]

    ya = (y_ + 1e-35) / a_
    return -1 * torch.mean(u_ * (torch.log(b_) + b_ * torch.log(ya)) - torch.pow(ya, b_))


def activate(ab):
    a = torch.exp(ab[:, 0])
    b = f.softplus(ab[:, 1])

    a = torch.reshape(a, (a.size()[0], 1))
    b = torch.reshape(b, (b.size()[0], 1))
    return torch.cat((a, b), axis=1)


def load_file(name):
    with open(name, 'r') as file:
        return np.loadtxt(file, delimiter=',')

class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()

        self.rnn = nn.LSTM(         # if use nn.RNN(), it hardly learns
            input_size=INPUT_SIZE,
            hidden_size=20,         # rnn hidden unit
            num_layers=2,           # number of rnn layer
            batch_first=True,       # input & output will has batch size as 1s dimension. e.g. (batch, time_step, input_size)
        )
        self.out = nn.Linear(20, 2)

    def forward(self, x):
        # x shape (batch, time_step, input_size)
        # r_out shape (batch, time_step, output_size)
        # h_n shape (n_layers, batch, hidden_size)
        # h_c shape (n_layers, batch, hidden_size)
        r_out, (h_n, h_c) = self.rnn(x, None)   # None represents zero initial hidden state

        # choose r_out at the last time step
        out = self.out(r_out[:, -1, :])
        return activate(out)

rnn = RNN()

optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)  
loss_func = weibull_loglik_discrete                       

np.set_printoptions(suppress=True, threshold=10000)

train = load_file('train.csv')
test_x = load_file('test_x.csv')
test_y = load_file('test_y.csv')


all_x = np.concatenate((train[:, 2:26], test_x[:, 2:26]))
all_x = normalize(all_x, axis=0)

train[:, 2:26] = all_x[0:train.shape[0], :]
test_x[:, 2:26] = all_x[train.shape[0]:, :]

train[:, 0:2] -= 1
test_x[:, 0:2] -= 1

max_time = 100

def build_data(engine, time, x, max_time, is_test):
    out_y = np.empty((0, 2), dtype=np.float32)
    out_x = np.empty((0, max_time, 24), dtype=np.float32)

    for i in range(100):
        print("Loading = " + str(i))
        max_engine_time = int(np.max(time[engine == i])) + 1

        if is_test:
            start = max_engine_time - 1
        else:
            start = 0

        this_x = np.empty((0, max_time, 24), dtype=np.float32)

        for j in range(start, max_engine_time):
            engine_x = x[engine == i]

            out_y = np.append(out_y, np.array((max_engine_time - j, 1), ndmin=2), axis=0)

            xtemp = np.zeros((1, max_time, 24))
            xtemp[:, max_time-min(j, 99)-1:max_time, :] = engine_x[max(0, j-max_time+1):j+1, :]
            this_x = np.concatenate((this_x, xtemp))

        out_x = np.concatenate((out_x, this_x))

    return out_x, out_y

train_x, train_y = build_data(train[:, 0], train[:, 1], train[:, 2:26], max_time, False)
test_x = build_data(test_x[:, 0], test_x[:, 1], test_x[:, 2:26], max_time, True)[0]

train_u = np.zeros((100, 1), dtype=np.float32)
train_u += 1
test_y = np.append(np.reshape(test_y, (100, 1)), train_u, axis=1)

EPOCH = 200

for epoch in range(EPOCH):
    b_x = torch.Tensor(train_x)              # reshape x to (batch, time_step, input_size)
    b_y = torch.Tensor(train_y)                               # batch y

    output = rnn(b_x)                               # rnn output
    loss = loss_func(output, b_y)                   # loss
    optimizer.zero_grad()                           # clear gradients for this training step
    loss.backward()                                 # backpropagation, compute gradients
    optimizer.step()                                # apply gradients
    print("Epoch = {}".format(epoch+1))

Loading = 0
Loading = 1
Loading = 2
Loading = 3
Loading = 4
Loading = 5
Loading = 6
Loading = 7
Loading = 8
Loading = 9
Loading = 10
Loading = 11
Loading = 12
Loading = 13
Loading = 14
Loading = 15
Loading = 16
Loading = 17
Loading = 18
Loading = 19
Loading = 20
Loading = 21
Loading = 22
Loading = 23
Loading = 24
Loading = 25
Loading = 26
Loading = 27
Loading = 28
Loading = 29
Loading = 30
Loading = 31
Loading = 32
Loading = 33
Loading = 34
Loading = 35
Loading = 36
Loading = 37
Loading = 38
Loading = 39
Loading = 40
Loading = 41
Loading = 42
Loading = 43
Loading = 44
Loading = 45
Loading = 46
Loading = 47
Loading = 48
Loading = 49
Loading = 50
Loading = 51
Loading = 52
Loading = 53
Loading = 54
Loading = 55
Loading = 56
Loading = 57
Loading = 58
Loading = 59
Loading = 60
Loading = 61
Loading = 62
Loading = 63
Loading = 64
Loading = 65
Loading = 66
Loading = 67
Loading = 68
Loading = 69
Loading = 70
Loading = 71
Loading = 72
Loading = 73
Loading = 74
Loading = 75
Loading = 76
Loading =

In [0]:
test_predict = rnn(torch.Tensor(test_x))
test_predict = test_predict.detach().numpy()

In [0]:
import matplotlib.pyplot as plt
from scipy.special import gamma

def meanW(ab):
    return ab[0]*gamma(1+1/ab[1])
def medianW(ab):
    return ab[0]*(-np.log(.5))**(1/ab[1])
def stdW(ab):
    return ab[0]*np.sqrt(gamma(1+2/ab[1])-gamma(1+1/ab[1])**2)
def weibull(x, ab):
    a = ab[0]
    b = ab[1]
    y = np.copy(x)
    for i in range(len(x)):
        if x[i] > 0:
            y[i] = b/a*((x[i]/a)**(b-1))*np.e**(-(x[i]/a)**b)
        else:
            y[i] = 0
    return y

case = 21
x = np.linspace(0,test_predict[case][0]*2,1024)
plt.axvline(test_y[case][0])
y = weibull(x,test_predict[case])
plt.plot(x,y)
plt.show()

In [0]:
life = np.zeros(len(test_y))
for i in range(len(test_y)):
    life[i] = test_y[i][0]
basis = np.arange(len(life))+1
order = np.argsort(life)

predictions = np.zeros(len(test_predict))
accuracy = np.zeros(len(test_predict))
for i in range(len(test_predict)):
    predictions[i] = medianW(test_predict[i])
    accuracy[i] = stdW(test_predict[i])

plt.scatter(basis,life[order])
plt.errorbar(basis,predictions[order],accuracy[order])
plt.show()

What we are trying to do is to obtain a single value from a set of data. Implicitly is like if we censor the data using the failure event